In [1]:
import os
import pickle
import numpy as np
from scipy.io import loadmat

import scfw.cov_estimation as ce
from scfw.frank_wolfe import frank_wolfe
from scfw.scopt import scopt

In [2]:
data_folder = '../data/'
problem_name = 'Lymph'
results = {problem_name: {}}
results_folder = './results/'
results_file = os.path.join(results_folder, 'cov_estimation', problem_name + '.pckl')
if os.path.isfile(results_file):
    with open(results_file, "rb") as f:
        try:
            results = pickle.load(f)
        except Exception:
            pass 

In [3]:
def sample_l1_point(r, ndim):
    vec = np.random.rand(ndim,)
    vec /= np.linalg.norm(vec, ord=1)
    return vec

In [4]:
# load data
data = loadmat(os.path.join(data_folder, problem_name + '.mat'))
    
A = data['S']
p = A.shape[0]
r = 1300
X_0 = r * np.eye(p) / p
#X_0 = np.diag(sample_l1_point(r, p))

## Parameters initialization

In [5]:
#first set of parameters
Mf = 2
nu = 3
#running parameters

terminate_tol = 1e-10

FW_params={
    'iter_FW': 100,
    'line_search_tol': 1e-5,
}


sc_params={
    #parameters for SCOPT
    'iter_SC': 1000,
    'Lest': 'estimate', #estimate L
    'use_two_phase': True,
    #FISTA parameters
    'fista_type': 'mfista',
    'fista_tol': 1e-5,
    'fista_iter': 1000,
    #Conjugate Gradient Parameters
    'conj_grad_tol': 1e-2,
    'conj_grad_iter': 100,
}

## Functions

In [6]:
func_x = lambda X: ce.cov_val(A, X)
func_beta = lambda X, S, beta, extra_param, extra_param_s: ce.cov_val(A, (1 - beta) * X + beta * S)
grad_x = lambda X, inv_X: ce.cov_grad(A, X, inv_X)
grad_beta = lambda X, S, beta, inv_X, inv_S: ce.cov_grad(A, (1 - beta) * X + beta * S, (1 - beta) * inv_X + beta * inv_S)
hess_mult_x = lambda S, inv_X: ce.cov_hess_mult(S, inv_X)
hess_mult_vec_x = lambda S, inv_X: ce.cov_hess_mult_vec(S, inv_X)
#extra_func = lambda X: np.linalg.eigvalsh(X)
extra_func = lambda X: X
linear_oracle = lambda grad: ce.linear_oracle(grad, r)
prox_func = lambda X, L: ce.proj_map(X) #used for SCOPT

## Run FW

In [7]:
#run_alpha_policies = ['standard', 'line_search', 'backtracking', 'sc', 'sc_backtracking', 'sc_hybrid']
run_alpha_policies = ['line_search']

for policy in run_alpha_policies:
    x, alpha_hist, Gap_hist, Q_hist, time_hist  = frank_wolfe(func_x,
                       func_beta,                                       
                       grad_x,
                       grad_beta,
                       hess_mult_x,
                       extra_func,                                                    
                       Mf,
                       nu,
                       linear_oracle,                                                    
                       X_0,
                       FW_params,
                       #hess=hess_x, 
                       #lloo_oracle=llo_oracle,                                                 
                       alpha_policy=policy,                                                    
                       eps=terminate_tol, 
                       print_every=10, 
                       debug_info=False)
    
    results[problem_name + '.mat'][policy] = {
        'x': x,
        'alpha_hist': alpha_hist,
        'Gap_hist': Gap_hist,
        'Q_hist': Q_hist,
        'time_hist': time_hist,
    }
#    with open('./results/cov_estimation/lymph.pckl', 'wb') as f:
#        pickle.dump(results, f)

********* Algorithm starts *********
beta_max: 0.003396
t_ub: 1.000000
iter=1, stepsize=1.00e+00, criterion=1.00e+00, upper_bound=833.279397234206, lower_bound=-1177.7958435120934, real_Gap=2.01e+03, f_val=833.279397234206


LinAlgError: Matrix is not positive definite

In [10]:
with open('./results/cov_estimation/lymph.pckl', 'wb') as f:
    pickle.dump(results, f)

In [7]:
x, alpha_hist, Gap_hist, Q_hist, time_hist  = frank_wolfe(func_x,
                       func_beta,                                       
                       grad_x,
                       grad_beta,
                       hess_mult_x,
                       extra_func,                                                    
                       Mf,
                       nu,
                       linear_oracle,                                                    
                       X_0,
                       FW_params,
                       #hess=hess_x, 
                       #lloo_oracle=llo_oracle,                                                 
                       alpha_policy='line_search',                                                    
                       eps=terminate_tol, 
                       print_every=1, 
                       debug_info=False)
                       

********* Algorithm starts *********
min_eig: -293.500000
beta_max: 0.003396
0.0033889536353396274
-652.2146507666092
iter=1, stepsize=3.39e-03, criterion=5.82e-02, upper_bound=833.279397234206, lower_bound=-1177.7958435120934, real_Gap=2.01e+03, f_val=833.279397234206
min_eig: 0.000000
beta_max: 1.000000
0.9999923705054695
-2.2108780343174272
iter=2, stepsize=1.00e+00, criterion=5.82e-02, upper_bound=832.010083035731, lower_bound=-1177.7958435120934, real_Gap=2.01e+03, f_val=832.010083035731
min_eig: 0.000000
beta_max: 1.000000
0.9999923705054695
-1299.9900984965147
iter=3, stepsize=1.00e+00, criterion=5.82e-02, upper_bound=832.010083035731, lower_bound=-1177.7958435120934, real_Gap=2.01e+03, f_val=7734.018144452073
min_eig: 0.000000
beta_max: 1.000000
0.9999923705054695
-1299.9900816572379
iter=4, stepsize=1.00e+00, criterion=5.82e-02, upper_bound=832.010083035731, lower_bound=-1177.7958435120934, real_Gap=2.01e+03, f_val=14620.980842775627
min_eig: 0.000000
beta_max: 1.000000
0.9999

KeyboardInterrupt: 

In [12]:
min_eig, _ = scipy.linalg.eigh(X_0)
print(min(min_eig))

2.2146507666098807


In [11]:
import scipy

In [23]:
a = np.array([[0, -10, 0], [-10, 0, 0], [0, 0, 0]])

In [24]:
min_eig, _ = scipy.linalg.eigh(a)
print(min(min_eig))

-9.99999999999999
